In [ ]:
!pip install tensorflow tensorflow-hub tensorflow-io librosa matplotlib pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 64.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm


/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutex6unlockEv']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.12/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZN3tsl7strings13safe_strtou64ESt17basic_string_viewIcSt11char_traitsIcEEPm']
  warnings.warn(

In [ ]:
model_url = "https://tfhub.dev/google/bird-vocalization-classifier/1"
model = hub.load(model_url)
print("✅ Model loaded!")

✅ Model loaded!


In [ ]:
labels_path = hub.resolve(model_url) + "/assets/label.csv"
labels_df = pd.read_csv(labels_path)
labels = labels_df["ebird2021"].tolist()
print(f"✅ Loaded {len(labels)} labels")

✅ Loaded 10932 labels


In [ ]:
# --- Load eBird taxonomy for readable species names ---
!curl -s -O https://www.birds.cornell.edu/clementschecklist/wp-content/uploads/2021/08/eBird_Taxonomy_v2021.csv

taxonomy = pd.read_csv("eBird_Taxonomy_v2021.csv")

print("Taxonomy columns:", taxonomy.columns.tolist())
code_col = "SPECIES_CODE"



# Merge the classifier labels with the taxonomy info
labels_df = labels_df.merge(taxonomy, left_on="ebird2021", right_on=code_col, how="left")

# Choose human-readable names if available
labels_human = labels_df["PRIMARY_COM_NAME"].fillna(labels_df["ebird2021"]).tolist()

print(f"✅ Loaded {len(labels_human)} human-readable labels")


🔎 Taxonomy columns: ['TAXON_ORDER', 'CATEGORY', 'SPECIES_CODE', 'PRIMARY_COM_NAME', 'SCI_NAME', 'ORDER1', 'FAMILY', 'SPECIES_GROUP', 'REPORT_AS']
✅ Loaded 10932 human-readable labels


In [ ]:
# --- Audio loader ---
def load_audio(path, target_sample_rate=32000):
    """Load and resample an audio file."""
    audio, sr = librosa.load(path, sr=None, mono=True)
    if sr != target_sample_rate:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sample_rate)
    return audio

# --- Classify a single 5-second clip ---
def classify_audio(model, audio, labels, top_k=5):
    """Run the classifier on a short clip and return top predictions."""
    # Trim or pad to 160000 samples (5 seconds at 32kHz)
    target_len = 160000
    if len(audio) < target_len:
        audio = np.pad(audio, (0, target_len - len(audio)))
    else:
        audio = audio[:target_len]

    audio = tf.convert_to_tensor(audio, dtype=tf.float32)
    audio = tf.reshape(audio, [1, 160000])

    outputs = model.signatures['serving_default'](inputs=audio)
    probs = tf.nn.softmax(outputs['output_0'][0])
    top_indices = tf.argsort(probs, direction="DESCENDING")[:top_k].numpy()

    return [(labels[i], float(probs[i])) for i in top_indices]

# --- Classify an entire long file in 5-second windows ---
def classify_long_audio_per_segment(model, audio, labels, sample_rate=32000, window_sec=5, top_k=5):
    """Split long audio into 5s chunks and classify each independently."""
    window_len = sample_rate * window_sec
    segments = len(audio) // window_len
    results = []

    for i in range(segments):
        start = i * window_len
        end = start + window_len
        clip = audio[start:end]
        preds = classify_audio(model, clip, labels, top_k)
        results.append((i, preds[0]))  # store (segment index, top-1 prediction)

    return results

In [ ]:
# --- Run classifier on your Drive folder ---
AUDIO_FOLDER = "/content/drive/MyDrive/Audio_Data"

for file_name in tqdm(os.listdir(AUDIO_FOLDER)):
    if file_name.lower().endswith(('.wav', '.ogg', '.mp3')):
        file_path = os.path.join(AUDIO_FOLDER, file_name)
        print(f"\n🎧 {file_name}")
        audio = load_audio(file_path)

        segment_results = classify_long_audio_per_segment(model, audio, labels_human)
        for seg_idx, (species, prob) in segment_results:
            print(f"  Segment {seg_idx:02d}: {species:<35} {prob:.2%}")

  0%|          | 0/1 [00:00<?, ?it/s]


🎧 sample.wav


100%|██████████| 1/1 [01:05<00:00, 65.27s/it]

  Segment 00: Black Phoebe                        74.37%
  Segment 01: Black Phoebe                        87.41%
  Segment 02: Black Phoebe                        79.59%
  Segment 03: Great Horned Owl                    96.50%
  Segment 04: Great Horned Owl                    92.32%
  Segment 05: Song Sparrow                        98.86%
  Segment 06: Song Sparrow                        90.20%
  Segment 07: Great Horned Owl                    90.35%
  Segment 08: Song Sparrow                        89.42%
  Segment 09: Song Sparrow                        96.73%
  Segment 10: Song Sparrow                        97.97%
  Segment 11: Song Sparrow                        81.72%
  Segment 12: Song Sparrow                        98.00%
  Segment 13: Song Sparrow                        92.72%
  Segment 14: Song Sparrow                        97.69%
  Segment 15: Song Sparrow                        87.23%
  Segment 16: Great Horned Owl                    98.17%
  Segment 17: Song Sparrow     